<a href="https://colab.research.google.com/github/bmill42/musical-structure/blob/main/Computing_with_twelve_tones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

The first two cells install some libraries and define some utility functions that will let us do things like see and hear MIDI versions of the musical objects we create. Start reading at the next heading.

In [ ]:
!pip install music21 --quiet
!pip install midi_player --quiet

In [ ]:
import random
import music21
from midi_player import MIDIPlayer
from midi_player.stylers import basic, cifka_advanced
from fractions import Fraction

def even_print(thing, space=2):
    print(' '.join([str(i) + (space - len(str(i))) * ' ' for i in thing]).strip())
    return None

def pcset_to_string(pcset, dur='quarter', wait=1, offset=60):
    return ' '.join(['n_{}_{} w_{}.0'.format(str(offset+p), dur, wait) for p in pcset])

def string_to_midi(pcset, dur='quarter', wait=1, offset=60):
    s = pcset_to_string(pcset, dur, wait, offset)
    stream = music21.stream.Stream()
    time = 1
    for i in s.split():
        if i.startswith('n'):
            note, duration = i.lstrip('n_').split('_')
            n = music21.note.Note(int(note))
            n.duration.type = duration
            stream.insert(time, n)
        elif i.startswith('w'):
            time += float(Fraction(i.lstrip('w_')))
    return stream

def play_midi(pcset, dur='quarter', wait=1, offset=60):
    midi = string_to_midi(pcset, dur, wait, offset)
    midi.write('midi', 'generated.midi')
    return MIDIPlayer('generated.midi', 160, styler=cifka_advanced, title='My Player', width='50%')

# Pitch and pitch class

For the most part we will use numbers to refer to pitches instead of the typical letters. For example, 60 is the MIDI number for the middle C key on the piano.

A musical interval is simply the distance in pitch between two notes. We'll begin by thinking of intervals in terms of keys on the piano, both black and white. For example, the notes 60 and 65 represent an interval of 5 keys, or semitones.

Producing these basic **pitch intervals** on the computer is very simple: *just add the size of the interval to the number for the starting pitch.*

## Exercise: Pitch intervals

**Fill out this function** to return a list containing the starting pitch and the pitch separated from it by the given interval.

In [ ]:
def pitch_interval(start_pitch, interval=0):
    pcset = # your code here
    return pcset

If the function above is working, you can plug in the starting pitch and interval here and see and hear the results in the MIDI player.

In [ ]:
starting = 60 # Set these variables to try out different pitch intervals
interval = 15

play_midi(pitch_interval(starting, interval), dur='half', wait=2, offset=0)

## Pitch class

**Pitch class** is a name for all of the pitches that are a whole number of octaves apart. For example, the piano has keys named for the pitches C1, C2, C3, C4, and so on, but all of these share the pitch class C.

Pitch class can be thought of as the "generic" name for any given pitch; it can also be thought of as the result of reducing all pitches down to the same octave.

Since we're using numbers instead of letters to represent pitches, and the Western equal-tempered octave has 12 notes, then adding or subtracting 12 (or any multiple of 12) gives us the same pitch class in a different octave.

For example, pitch number 60 is right in the middle of the piano keyboard. Adding `12 * n` to 60 shifts the pitch by `n` octaves but maintains the same pitch class.

In [ ]:
n = 2 # number of octaves to shift the pitch
60 + (12 * n) # this statement will always return pitch numbers with the same pitch class

Going in the other direction, we can reduce any actual pitch down to its abstract pitch class by taking its number modulo 12 (recall the clock face example). The note C is traditionally labeled as 0, so shifting note 60 by any number of octaves will always leave us with pitch class 0.

In [ ]:
(60 + (12 * n)) % 12

### The pitch class universe

We can generate a list of all the pitch classes using the `range()` function in Python, which simply returns a list counting from the first argument to the second.

Think of this list as representing a single octave on the piano - or *all* octaves on the piano at once.

In [ ]:
pcs = list(range(0,12))
pcs

## Pitch class intervals

Pitch class intervals are similar to pitch intervals, but both the starting and ending pitches are always within the range of 0 to 11.

Think of a clock that starts at midnight, then the same clock at 3pm on the same day. The "real" time interval is 15 hours, but the hand on the clock looks like it's only moved 3 hours ahead.

The math to produce pitch class intervals is the same as the math to produce single pitch classes: just take the result of the pitch interval modulo 12.

In [ ]:
(0 + 15) % 12

### Exercise: calculating pitch class intervals

**Fill out this function** to return a list containing the starting pitch class and the pitch class separated from it by the given interval.

In [ ]:
def pc_interval(start_pc, interval=0):
    pcset = # your code here

    return pcset

You can enter the starting pitch class and interval here to listen to the results. Can you find a combination that results in a downwards leap despite being a positive interval?

In [ ]:
starting = 6 # Set these variables to try out different pc intervals
interval = 8

play_midi(pc_interval(starting, interval), dur='half', wait=2)

# Twelve-tone music

As we've seen, a major current in experimental classical music starting in the early twentieth century was the effort to escape from the existing tonal system, which privileges a set of seven out of the twelve pitch classes, by making equal use of all twelve.

A *twelve-tone* or *serialist* work is typically based on a single **tone row**: a set of all twelve pitch classes in any order.

We can generate a random tone row from our initial pitch class universe using Python's `random.shuffle()` method.

In [ ]:
tone_row = list(range(0,12))

random.shuffle(tone_row)
tone_row

Listen to your melodious new tone row here.

In [ ]:
play_midi(tone_row)

## Exercise: calculate a tone row's intervals

The numbers in our tone row represent pitch classes, but the characteristic sound of a tone row comes from the *relationships* among these notes - in other words, the **intervals** between the pitch classes.

For example, the first interval in the row is the distance between the first and second pitch classes; the second interval is the distance between the second and third pitch class, etc.

**Fill out this cell** to create a new Python list called `intervals` containing the pitch class interval between each pair of notes (wrapping around from the last to the first, for 12 total numbers). *Every interval should be a positive number ranging from 1 to 11*. This can be done in one line of code but might be easier with a `for` loop.

In [ ]:
intervals = []

for pc in tone_row:
    interval = # your code here

    intervals.append(interval)

print(intervals)

## The twelve-tone matrix

One of the classic tools of twelve-tone composition is the *twelve-tone matrix*. This is a sudoku-like 12x12 grid that begins with the original tone row on the first line.

The key to the matrix is the set of intervals that we already calculated. Going downwards from the top left, we repeat the tone row, but **inverted**, meaning that we follow the same set of intervals but negate each of them first. If the first interval is 2, the first inverted interval is -2, and so on.

Each column then repeats the same set of intervals, *starting from each successive pitch in the original row*.

In the resulting table, the rows are called the **prime forms (P)** of the tone row, because they represent the same set of intervals as the main row but starting on a different pitch. The columns are the **inverted forms (I)**.

Reading each row backwards gives the **retrograde forms (R)**, again starting on each pitch, and reading the columns from bottom to top gives the **retrograde inverted forms (RI)**.

Together, these four forms represent an array of materials that a composer could use to write a twelve-tone piece.

There are various ways to do the math that generates the matrix; this code does it by generating the columns going downward using the inverted intervals.

In [ ]:
for i in range(12):
    even_print([(x - sum(intervals[0:i])) % 12 for x in tone_row], 3)

The matrix shows us 12 forms each of **P**, **I**, **R**, and **RI**.

One way we could extract all of these forms would be to turn this matrix into a dataframe and pull out individual rows or columns as needed.

But instead we'll write functions that can take in the original prime form of a row and give us one of the 47 other possible forms by manipulating the prime form mathematically.

### Exercise: transposition

**Transposition** is the musical process of shifting a musical object (a note, a chord, a melody, or a tone row) by some interval.

If our musical object is a list of notes, we simply add or subtract the same interval to every note in the list.

**Fill in this function** so that it takes in a pitch class set (abbreviated `pcset`) and an interval and returns the transposed set as a new list.

In [ ]:
def transpose(pcset, interval):
    transposed = # your code here
    return transposed

In [ ]:
transpose(tone_row, 9)

### Exercise: inversion

**Inverting** a musical object means mirroring its intervals, usually around the starting pitch. This is the same process we observed in the first column of the twelve-tone matrix, where we start from the first pitch of the original row and subtract, rather than add, its intervals.

**Fill in this function** so that it takes in a `pcset` and returns an inverted set that *begins on the same pitch class*. The code you wrote before to generate the tone row's intervals will be useful here.

In [ ]:
def invert(pcset):
    inverted = # your code here

    return inverted

In [ ]:
invert(tone_row)

### Non-exercise: retrograde



The **retrograde** operation is the easiest to perform - you simply play the musical object's pitches in reverse order.

I'll give you this one:

In [ ]:
def retrograde(pcset):
    return list(reversed(pcset))

In [ ]:
retrograde(tone_row)

### Exercise: P, R, I, and RI forms



The classic twelve-tone matrix includes labels around all its sides identifying the P, R, I, or RI form associated with each row or column. Let's write a function that can generate any of these on command by combining the functions we already wrote.

**Fill in this function** to produce any of the 48 forms of the given tone row. The function will take in an argument called `form` that consists of a short string (`'p', 'r', 'i', or 'ri'`) and the number of the pitch class the resulting row should start on.

In [ ]:
def pri(pcset, form='p', transp=0):
    pcset_pri = # your code here

    return pcset_pri

In [ ]:
transformed = pri(tone_row, 'ri', 7)
print(transformed)
play_midi(transformed)